In [1]:
import os
import subprocess
import pandas as pd
from tqdm.notebook import tqdm

DATA_FOLDER = '../data/'

# Beer Advocate folder:

In [3]:
beers = pd.read_csv('../data/BeerAdvocate/beers.csv')
breweries = pd.read_csv('../data/BeerAdvocate/breweries.csv')
users = pd.read_csv('../data/BeerAdvocate/users.csv')

In [5]:
print(beers.shape)
print(beers.beer_id.nunique())

print(breweries.shape)
print(breweries.id.nunique())

print(users.shape)
print(users.user_id.nunique())

(280823, 15)
280823
(16758, 4)
16758
(153704, 6)
153704


In [57]:
beers.head(1)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.7,NaN,NaN,0,NaN


In [9]:
beers['style'].unique()

array(['Euro Pale Lager', 'American Pale Lager', 'Euro Dark Lager',
       'Foreign / Export Stout', 'Irish Red Ale', 'Czech Pilsener',
       'Kölsch', 'Dortmunder / Export Lager', 'Munich Dunkel Lager',
       'Märzen / Oktoberfest', 'English Pale Ale', 'English Bitter',
       'American Pale Wheat Ale', 'American Blonde Ale',
       'English India Pale Ale (IPA)', 'Irish Dry Stout',
       'American Stout', 'American Pale Ale (APA)', 'Milk / Sweet Stout',
       'American Double / Imperial IPA', 'German Pilsener',
       'American IPA', 'Witbier', 'Munich Helles Lager',
       'English Brown Ale', 'English Porter', 'English Pale Mild Ale',
       'American Black Ale', 'Saison / Farmhouse Ale',
       'Berliner Weissbier', 'American Double / Imperial Stout',
       'American Porter', 'American Wild Ale', 'Fruit / Vegetable Beer',
       'English Dark Mild Ale', 'Hefeweizen', 'English Stout',
       'California Common / Steam Beer', 'American Amber / Red Ale',
       'Tripel', 'Altbie

In [58]:
breweries.head(1)

,id,location,name,nbr_beers
0,39912,Kyrgyzstan,Abdysh-Ata (Абдыш Ата),5


In [12]:
users.head(5)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland
3,31,31,helloloser12345.10867,helloloser12345,1.101380e+09,Northern Ireland
4,604,604,cypressbob.3708,cypressbob,1.069326e+09,Northern Ireland


In [13]:
!head -n 20 /Users/tommery/Desktop/MA3/ADA_TooHotToHoot/ada-2022-project-toohottohoot/data/BeerAdvocate/ratings.txt

beer_name: Régab
beer_id: 142544
brewery_name: Societe des Brasseries du Gabon (SOBRAGA)
brewery_id: 37262
style: Euro Pale Lager
abv: 4.5
date: 1440064800
user_name: nmann08
user_id: nmann08.184925
appearance: 3.25
aroma: 2.75
palate: 3.25
taste: 2.75
overall: 3.0
rating: 2.88
text: From a bottle, pours a piss yellow color with a fizzy white head.  This is carbonated similar to soda.The nose is basic.. malt, corn, a little floral, some earthy straw.  The flavor is boring, not offensive, just boring.  Tastes a little like corn and grain.  Hard to write a review on something so simple.Its ok, could be way worse.
review: True

beer_name: Barelegs Brew
beer_id: 19590


In [14]:
!head -n 20 /Users/tommery/Desktop/MA3/ADA_TooHotToHoot/ada-2022-project-toohottohoot/data/BeerAdvocate/reviews.txt

beer_name: Régab
beer_id: 142544
brewery_name: Societe des Brasseries du Gabon (SOBRAGA)
brewery_id: 37262
style: Euro Pale Lager
abv: 4.5
date: 1440064800
user_name: nmann08
user_id: nmann08.184925
appearance: 3.25
aroma: 2.75
palate: 3.25
taste: 2.75
overall: 3.0
rating: 2.88
text: From a bottle, pours a piss yellow color with a fizzy white head.  This is carbonated similar to soda.The nose is basic.. malt, corn, a little floral, some earthy straw.  The flavor is boring, not offensive, just boring.  Tastes a little like corn and grain.  Hard to write a review on something so simple.Its ok, could be way worse.

beer_name: Barelegs Brew
beer_id: 19590
brewery_name: Strangford Lough Brewing Company Ltd


### Ratings.txt and reviews.txt to dataframe

In [8]:
def parse_txt_file(path, features, nb_lines):
    
    file = open(path)
    
    d = {}
    ds = []
    
    for line in tqdm(file, desc="Parsing {} file: ".format(path.split('/')[-1]), total=nb_lines):
        if line == '\n':            
            ds.append(d)
            d = {}
            continue
        
        key, val = line.split(':', maxsplit=1)
        if key not in features:
            continue
        d[key] = val[1:-1]
    
    return pd.DataFrame(ds)

### Differences between ratings.txt and reviews.txt

In [25]:
!wc -l < ../data/BeerAdvocate/ratings.txt
!wc -l < ../data/BeerAdvocate/reviews.txt

 151074576
 44022962


In [ ]:
nb_lines_ratings = !wc -l < '../data/BeerAdvocate/ratings.txt'
nb_lines_ratings = int(nb_lines_ratings[0])

nb_lines_reviews = !wc -l < '../data/BeerAdvocate/reviews.txt'
nb_lines_reviews = int(nb_lines_reviews[0])

ratings = parse_txt_file(DATA_FOLDER+'BeerAdvocate/ratings.txt', features=['review'], nb_lines=nb_lines_ratings)
reviews = parse_txt_file(DATA_FOLDER+'BeerAdvocate/reviews.txt', features=['beer_name'], nb_lines=nb_lines_reviews)

nb_text_in_ratings = ratings['review'].apply(lambda x: 1 if x == 'True' else 0).sum()
print(nb_text_in_ratings == len(reviews))

Conclusion: It seems to be reasonable to consider that all review in `reviews.txt` are in `ratings.txt`, since the number of attribute **review** equal to **True** in `ratings.txt` correspond to the number of actual review in `reviews.txt`. Therefore taking only `ratings.txt` does not leave any information. 

### Check constraints in relational model

In [9]:
nb_lines_ratings = !wc -l < '../data/BeerAdvocate/ratings.txt'
nb_lines_ratings = int(nb_lines_ratings[0])

ratings = parse_txt_file(DATA_FOLDER+'BeerAdvocate/ratings.txt', features=['beer_id', 'user_id', 'date'], nb_lines=nb_lines_ratings)

Parsing ratings.txt file:   0%|          | 0/151074576 [00:00<?, ?it/s]

In [19]:
# Participation and key constraints for 'Ratings' relationship:
print(ratings['beer_id'].nunique(), beers.beer_id.nunique())    # Partial participation on beers entity
print(ratings['user_id'].nunique(), users.user_id.nunique())    # Full participation on user entity
print(ratings['beer_id'].nunique(), beers.beer_id.nunique())    # Partial participation on beers entity

247982 280823
153704 153704
247982 280823


In [14]:
# Participation and key constraints for 'Produced_by' relationship:
print(beers.brewery_id.isnull().sum())                  # Full participation + key constraint on beers entity
print(beers.brewery_id.nunique(), breweries.shape[0])   # Partial participation on breweries entity

0
14325 16758


# Rate Beer folder:

In [20]:
beers = pd.read_csv('../data/RateBeer/beers.csv')
breweries = pd.read_csv('../data/RateBeer/breweries.csv')
users = pd.read_csv('../data/RateBeer/users.csv')

In [61]:
beers.head(1)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,overall_score,style_score,avg,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,410549,33 Export (Gabon),3198,Sobraga,Pale Lager,1,NaN,NaN,2.72,5.0,2.0,NaN,0,NaN


In [62]:
breweries.head(1)

,id,location,name,nbr_beers
0,3198,Gabon,Sobraga,3


In [63]:
users.head(1)

,nbr_ratings,user_id,user_name,joined,location
0,1890,175852,Manslow,1.337508e+09,Poland


In [64]:
!head -n 20 /Users/tommery/Desktop/MA3/ADA_TooHotToHoot/ada-2022-project-toohottohoot/data/RateBeer/ratings.txt

beer_name: 33 Export (Gabon)
beer_id: 410549
brewery_name: Sobraga
brewery_id: 3198
style: Pale Lager
abv: 5.0
date: 1461664800
user_name: Manslow
user_id: 175852
appearance: 2
aroma: 4
palate: 2
taste: 4
overall: 8
rating: 2.00
text: Puszka 0,33l dzięki Christoph . Kolor jasnozłoty, piana biała drobnoziarnista szybko opada. Aromat słodowy, lekko słodkie.Dość dobre. 

beer_name: Castel Beer (Gabon)
beer_id: 105273
brewery_name: Sobraga


In [65]:
!head -n 20 /Users/tommery/Desktop/MA3/ADA_TooHotToHoot/ada-2022-project-toohottohoot/data/RateBeer/reviews.txt

beer_name: 33 Export (Gabon)
beer_id: 410549
brewery_name: Sobraga
brewery_id: 3198
style: Pale Lager
abv: 5.0
date: 1461664800
user_name: Manslow
user_id: 175852
appearance: 2
aroma: 4
palate: 2
taste: 4
overall: 8
rating: 2.00
text: Puszka 0,33l dzięki Christoph . Kolor jasnozłoty, piana biała drobnoziarnista szybko opada. Aromat słodowy, lekko słodkie.Dość dobre. 

beer_name: Castel Beer (Gabon)
beer_id: 105273
brewery_name: Sobraga


### Differences between ratings.txt and reviews.txt

In [3]:
!wc -l < ../data/RateBeer/ratings.txt
!wc -l < ../data/RateBeer/reviews.txt

 121075258
 121075258


In [2]:
with open('../data/RateBeer/ratings.txt') as f:
    lines_ratings = f.readlines()

with open('../data/RateBeer/reviews.txt') as f:
    lines_reviews = f.readlines()

for l1, l2 in tqdm(zip(lines_ratings, lines_reviews), total=len(lines_ratings), desc="Checking if 'ratings.txt' and 'reviews.txt' in RateBeer folder are different"):
    if l1 != l2:
        raise ValueError('Files are different')

print("'ratings.tx' and 'reviews.txt' files in RateBeer folder are the same.")

Checking if 'ratings.txt' and 'reviews.txt' in RateBeer folder are different:   0%|          | 0/121075258 [00…

'ratings.tx' and 'reviews.txt' files in RateBeer folder are the same.


### Check constraints in relational model

In [24]:
nb_lines_ratings = !wc -l < '../data/RateBeer/ratings.txt'
nb_lines_ratings = int(nb_lines_ratings[0])

ratings = parse_txt_file(DATA_FOLDER+'RateBeer/ratings.txt', features=['beer_id', 'user_id', 'date'], nb_lines=nb_lines_ratings)

Parsing ratings.txt file:   0%|          | 0/121075258 [00:00<?, ?it/s]

In [25]:
# Participation and key constraints for 'Ratings' relationship:
print(ratings['beer_id'].nunique(), beers.beer_id.nunique())    # Partial participation on beers entity
print(ratings['user_id'].nunique(), users.user_id.nunique())    # Full participation on user entity
print(ratings['beer_id'].nunique(), beers.beer_id.nunique())    # Partial participation on beers entity

396690 442081
70121 70120
396690 442081


In [26]:
# Participation and key constraints for 'Produced_by' relationship:
print(beers.brewery_id.isnull().sum())                  # Full participation + key constraint on beers entity
print(beers.brewery_id.nunique(), breweries.shape[0])   # Partial participation on breweries entity

0
23199 24189


# Matched beer data folder:

In [4]:
beers = pd.read_csv('../data/matched_beer_data/beers.csv')
breweries = pd.read_csv('../data/matched_beer_data/breweries.csv')
users = pd.read_csv('../data/matched_beer_data/users.csv')
users_approx = pd.read_csv('../data/matched_beer_data/users_approx.csv')
ratings = pd.read_csv('../data/matched_beer_data/ratings.csv')

/var/folders/y_/y88txvzd6wn04_qgbshn3ct80000gn/T/ipykernel_24229/2726088533.py:1: DtypeWarning: Columns (0,1,2,3,4,5,8,10,11,12,13,15,16,17,18,19,20,23,25,26,27,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  beers = pd.read_csv('../data/matched_beer_data/beers.csv')
/var/folders/y_/y88txvzd6wn04_qgbshn3ct80000gn/T/ipykernel_24229/2726088533.py:5: DtypeWarning: Columns (0,1,2,3,5,7,8,9,10,11,13,17,18,19,20,22,24,25,26,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv('../data/matched_beer_data/ratings.csv')


In [5]:
beers.head(1)

,ba,ba.1,ba.2,ba.3,ba.4,ba.5,ba.6,ba.7,ba.8,ba.9,...,rb.7,rb.8,rb.9,rb.10,rb.11,rb.12,rb.13,rb.14,scores,scores.1
0,abv,avg,avg_computed,avg_matched_valid_ratings,ba_score,beer_id,beer_name,beer_wout_brewery_name,brewery_id,brewery_name,...,brewery_id,brewery_name,nbr_matched_valid_ratings,nbr_ratings,overall_score,style,style_score,zscore,diff,sim


In [6]:
breweries.head(1)

,ba,ba.1,ba.2,ba.3,rb,rb.1,rb.2,rb.3,scores,scores.1
0,id,location,name,nbr_beers,id,location,name,nbr_beers,diff,sim


In [7]:
users.head(1)

,ba,ba.1,ba.2,ba.3,ba.4,ba.5,ba.6,rb,rb.1,rb.2,rb.3,rb.4,rb.5
0,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined,location,nbr_ratings,user_id,user_name,user_name_lower


In [8]:
users_approx.head(1)

,ba,ba.1,ba.2,ba.3,ba.4,ba.5,ba.6,rb,rb.1,rb.2,rb.3,rb.4,rb.5,scores
0,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined,location,nbr_ratings,user_id,user_name,user_name_lower,sim


In [9]:
ratings.head(1)

,ba,ba.1,ba.2,ba.3,ba.4,ba.5,ba.6,ba.7,ba.8,ba.9,...,rb.6,rb.7,rb.8,rb.9,rb.10,rb.11,rb.12,rb.13,rb.14,rb.15
0,abv,appearance,aroma,beer_id,beer_name,brewery_id,brewery_name,date,overall,palate,...,brewery_name,date,overall,palate,rating,style,taste,text,user_id,user_name


In [30]:
!head -n 20 /Users/tommery/Desktop/MA3/ADA_TooHotToHoot/ada-2022-project-toohottohoot/data/matched_beer_data/ratings_ba.txt

beer_name: Legbiter
beer_id: 19827
brewery_name: Strangford Lough Brewing Company Ltd
brewery_id: 10093
style: English Pale Ale
abv: 4.8
date: 1417431600
user_name: Hellpop65
user_id: hellpop65.48993
appearance: nan
aroma: nan
palate: nan
taste: nan
overall: nan
rating: 3.25
text: nan
review: False

beer_name: Legbiter
beer_id: 19827


In [31]:
!head -n 20 /Users/tommery/Desktop/MA3/ADA_TooHotToHoot/ada-2022-project-toohottohoot/data/matched_beer_data/ratings_rb.txt

beer_name: Ards Bally Black Stout
beer_id: 155699
brewery_name: Ards Brewing Co.
brewery_id: 13538
style: Stout
abv: 4.6
date: 1429178400
user_name: ciaranc
user_id: 151109
appearance: 3
aroma: 7
palate: 2
taste: 8
overall: 17
rating: 3.70
text: Bottle, gift from Aaron. Black, big head, lots of fizz. Head doesnât stick around. Roast, bitter coffee notes, roasted barley, light body, loads of carbonation. Decent flavour and very drinkable

beer_name: Ards Bally Black Stout
beer_id: 155699
brewery_name: Ards Brewing Co.


In [32]:
!head -n 20 /Users/tommery/Desktop/MA3/ADA_TooHotToHoot/ada-2022-project-toohottohoot/data/matched_beer_data/ratings_with_text_ba.txt

beer_name: Legbiter
beer_id: 19827
brewery_name: Strangford Lough Brewing Company Ltd
brewery_id: 10093
style: English Pale Ale
abv: 4.8
date: 1360753200
user_name: brewdude1982
user_id: brewdude1982.691169
appearance: 4.25
aroma: 4.5
palate: 3.25
taste: 3.75
overall: 3.75
rating: 3.91
text: This was a 12 oz bottle poured into a snifter.It pours a dark brown with some clarity but not truly transparent. It has a frothy head with mild retention and low lacing. By far the best thing this beer has going for it is a truly amazing malty aroma that does not dissapoint. If only the taste delivered the same punch. It has a mild malt taste and is followed by a metallic finish. This could be due to being Ina bottle so I would try again on draft but it did let me down. The body is kind of weak with decent carbonation. I would buy again if the price was right but it didn't strike me as awesome.
review: True

beer_name: Legbiter
beer_id: 19827


In [33]:
!head -n 20 /Users/tommery/Desktop/MA3/ADA_TooHotToHoot/ada-2022-project-toohottohoot/data/matched_beer_data/ratings_with_text_rb.txt

beer_name: Ards Bally Black Stout
beer_id: 155699
brewery_name: Ards Brewing Co.
brewery_id: 13538
style: Stout
abv: 4.6
date: 1429178400
user_name: ciaranc
user_id: 151109
appearance: 3
aroma: 7
palate: 2
taste: 8
overall: 17
rating: 3.70
text: Bottle, gift from Aaron. Black, big head, lots of fizz. Head doesnât stick around. Roast, bitter coffee notes, roasted barley, light body, loads of carbonation. Decent flavour and very drinkable

beer_name: Ards Bally Black Stout
beer_id: 155699
brewery_name: Ards Brewing Co.
